# Agentics Tutorial
This is a step by step guide illustrating how to make a new AG, access and print its content, import and export it to files

In [ ]:
! uv pip install agentics-py

import os
from pathlib import Path
import sys
from getpass import getpass

from dotenv import find_dotenv, load_dotenv

CURRENT_PATH = ""

IN_COLAB = "google.colab" in sys.modules
print("In Colab:", IN_COLAB)

if IN_COLAB:
    CURRENT_PATH = "/content/drive/MyDrive/"
    # Mount your google drive
    from google.colab import drive
    drive.mount("/content/drive")
    load_dotenv("/content/drive/MyDrive/.env")
else:
    load_dotenv(find_dotenv())

if not os.getenv("GEMINI_API_KEY"):
    os.environ["GEMINI_API_KEY"] = getpass("Enter your GEMINI_API_KEY:")

base = Path(CURRENT_PATH)

Using Python 3.12.9 environment at: /Users/gliozzo/Code/agentics911/agentics/.venv
Audited 1 package in 17ms
In Colab: False


## Define a new AG

Agentics provide the smarts of LLM to pydantic types. It does so by wrapping assigning any Pydantic type definitions to the atype field. 

In [2]:
from agentics.core.agentics import Agentics as AG
from pydantic import BaseModel
from typing import Optional

class Movie(BaseModel):
    title: Optional[str] = None
    genre: Optional[str] = None
    description:Optional[str] = None

movies = AG(atype=Movie)
print(movies.pretty_print())

2025-09-11 11:20:15.142 | DEBUG    | agentics.core.llm_connections:<module>:121 - AGENTICS is connecting to the following LLM API providers:
2025-09-11 11:20:15.143 | DEBUG    | agentics.core.llm_connections:<module>:124 - 0 - WatsonX
2025-09-11 11:20:15.143 | DEBUG    | agentics.core.llm_connections:<module>:129 - 1 - Gemini
2025-09-11 11:20:15.143 | DEBUG    | agentics.core.llm_connections:<module>:133 - 2 - OpenAI
2025-09-11 11:20:15.144 | DEBUG    | agentics.core.llm_connections:<module>:135 - Please add API keys in .env file to add or disconnect providers.
2025-09-11 11:20:15.150 | DEBUG    | agentics.core.llm_connections:get_llm_provider:29 - No LLM provider specified. Using the first available provider.
2025-09-11 11:20:15.150 | DEBUG    | agentics.core.llm_connections:get_llm_provider:31 - Available LLM providers: ['watsonx', 'gemini', 'openai']. Using 'watsonx'


Atype : <class '__main__.Movie'>



## Agentics are Typed Lists

In addition to atypes, the AG class allows us to store a list of pydantic objects which are instances of atype. This enable them to exibit a list like behaviour, which allows us to access their states as arrays and use the + operator for concatenation.


In [3]:
movies = AG(atype=Movie)
print(movies.states) ### Initially empty states

movies.states.append(Movie(title="The Godfather")) ### add new object
print(movies[0]) ## print the first state

movies.states.append(Movie(title="La dolce vita")) ## add second object
print(movies.pretty_print()) ## the pretty print function return a string describing atype and states of the agentic

for movie in movies:  ### Agentics can be used as lists and allow iteration on states
    print(movie.model_dump_json())

new_movies = AG(atype=Movie,states=[Movie(title="Superman")]) ### Creating a new agentics to show sum

movies += new_movies ## agentics of the same type can be concatenated using the + operand

print(movies.pretty_print())

print(len(movies))

[]
title='The Godfather' genre=None description=None
Atype : <class '__main__.Movie'>
title: The Godfather
genre: null
description: null

title: La dolce vita
genre: null
description: null


{"title":"The Godfather","genre":null,"description":null}
{"title":"La dolce vita","genre":null,"description":null}
Atype : <class '__main__.Movie'>
title: The Godfather
genre: null
description: null

title: La dolce vita
genre: null
description: null

title: Superman
genre: null
description: null


3


## Input/Output

Agentics states can be initialized loaded and saved to .csv or .jsonl files. AType will be automatically generated from the column names (after they will be normalized as needed for valid pydantic fields), with all attributes set to strings.


In [7]:
# Create a new AG object from the privided csv file
movies = AG.from_csv(base /"data/movies.csv",max_rows=10)

# Note that the type is automatically induced and all arguments are set to srt
print(movies.pretty_print()) 

# Export the same AG in jsonl format
movies.to_jsonl(base / "data/movies.jsonl")

# Load new agentic from jsonl
movies_copy = AG.from_jsonl(base / "data/movies.jsonl")
movies_copy += movies

# export the new AG in csv format
movies_copy.to_csv(base / "data/movies_copy.csv")

2025-09-11 11:21:54.849 | DEBUG    | agentics.core.agentics:to_jsonl:938 - Exporting 10 Agentics to CSV data/movies.jsonl
2025-09-11 11:21:54.853 | DEBUG    | agentics.core.agentics:to_csv:928 - Exporting 20 Agentics to CSV data/movies_copy.csv


Atype : <class 'agentics.core.utils.AType#movie_name:genre:descriptionOptional'>
movie_name: The Shawshank Redemption
genre: Drama, Crime
description: Imprisoned in the 1940s for the double murder of his wife and her lover,
  upstanding banker Andy Dufresne begins a new life at the Shawshank prison, where
  he puts his accounting skills to work for an amoral warden. During his long stretch
  in prison, Dufresne comes to be admired by the other inmates -- including an older
  prisoner named Red -- for his integrity and unquenchable sense of hope.

movie_name: The Godfather
genre: Drama, Crime
description: Spanning the years 1945 to 1955, a chronicle of the fictional Italian-American
  Corleone crime family. When organized crime family patriarch, Vito Corleone barely
  survives an attempt on his life, his youngest son, Michael steps in to take care
  of the would-be killers, launching a campaign of bloody revenge.

movie_name: The Godfather Part II
genre: Drama, Crime
description: In the